# Preparatory Processing and Calcs for the Heat Pump Calculator

In [75]:
import os
import csv
from datetime import datetime
import pickle

import pandas as pd
import numpy as np
from glob import glob
import requests
from IPython.display import display

## Download TMY3 Files

In [33]:
TMY_FOLDER = r'C:\Users\Alan\Dropbox\TMY3'
TMY_URL = 'http://rredc.nrel.gov/solar/old_data/nsrdb/1991-2005/data/tmy3/{}TYA.CSV'
#for lin in open(os.path.join(TMY_FOLDER, 'tmy_list.txt')):
#    tmy_id = lin.strip().split()[0]
#    print('Processing: {}'.format(tmy_id))
#    resp = requests.get(TMY_URL.format(tmy_id))
#    open(os.path.join(TMY_FOLDER, '{}.csv'.format(tmy_id)), 'w').write(resp.text)


## Process TMY3 Files into Pickled DataFrames

In [85]:
# accumulates metadata for each TMY3 file
meta_list = []

for f in glob(os.path.join(TMY_FOLDER, '*.csv')):
    print('Processing: {}'.format(f))
    with open(f, newline='') as csvfile:
        
        # Use a csvreader just to process the header row
        tmyreader = csv.reader(csvfile)
        hdr = next(tmyreader)
        meta = dict(
            tmy_id = int(hdr[0]),
            city = hdr[1].strip(),
            state = hdr[2].strip(),
            utc_offset = float(hdr[3]),
            latitude = float(hdr[4]),
            longitude = float(hdr[5]),
            elevation = float(hdr[6]) * 3.28084   # in feet
        )
        meta_list.append(meta)
        
        # read the rest of the lines into a DataFrame
        df = pd.read_csv(csvfile)
        
        # start making final DataFrame
        df['db_temp'] = df['Dry-bulb (C)'] * 1.8 + 32.0
        df_final = df[['db_temp']].copy()
        
        # make a list of date/times with the stamp occurring in the
        # middle of the hour associated with the data.  Also, use 
        # the year 2018 for all the timestamps
        ts = []
        for dt, tm in zip(df['Date (MM/DD/YYYY)'], df['Time (HH:MM)']):
            m, d, _ = dt.split('/')
            h, _ = tm.split(':')
            ts.append( datetime(2018, int(m), int(d), int(h) - 1, 30))
        
        df_final.index = ts
        df_final['month'] = df_final.index.month
        
        # pickle it.
        df_final.to_pickle('data/climate/{}.pkl'.format(meta['tmy_id']))

df_meta = pd.DataFrame(meta_list)
df_meta.set_index('tmy_id', inplace=True)
df_meta.to_pickle('data/climate/tmy3_meta.pkl')

Processing: C:\Users\Alan\Dropbox\TMY3\700197.csv
Processing: C:\Users\Alan\Dropbox\TMY3\700260.csv
Processing: C:\Users\Alan\Dropbox\TMY3\700637.csv
Processing: C:\Users\Alan\Dropbox\TMY3\701043.csv
Processing: C:\Users\Alan\Dropbox\TMY3\701195.csv
Processing: C:\Users\Alan\Dropbox\TMY3\701330.csv
Processing: C:\Users\Alan\Dropbox\TMY3\701625.csv
Processing: C:\Users\Alan\Dropbox\TMY3\701718.csv
Processing: C:\Users\Alan\Dropbox\TMY3\701740.csv
Processing: C:\Users\Alan\Dropbox\TMY3\701780.csv
Processing: C:\Users\Alan\Dropbox\TMY3\701940.csv
Processing: C:\Users\Alan\Dropbox\TMY3\702000.csv
Processing: C:\Users\Alan\Dropbox\TMY3\702005.csv
Processing: C:\Users\Alan\Dropbox\TMY3\702035.csv
Processing: C:\Users\Alan\Dropbox\TMY3\702040.csv
Processing: C:\Users\Alan\Dropbox\TMY3\702070.csv
Processing: C:\Users\Alan\Dropbox\TMY3\702075.csv
Processing: C:\Users\Alan\Dropbox\TMY3\702084.csv
Processing: C:\Users\Alan\Dropbox\TMY3\702185.csv
Processing: C:\Users\Alan\Dropbox\TMY3\702186.csv


In [84]:
display(df_final.head())
display(df_final.tail())
df_meta

,db_temp,month
2018-01-01 00:30:00,17.60,1
2018-01-01 01:30:00,19.58,1
2018-01-01 02:30:00,19.22,1
2018-01-01 03:30:00,16.34,1
2018-01-01 04:30:00,17.60,1


,db_temp,month
2018-12-31 19:30:00,-32.80,12
2018-12-31 20:30:00,-32.80,12
2018-12-31 21:30:00,-34.24,12
2018-12-31 22:30:00,-34.60,12
2018-12-31 23:30:00,-34.60,12


,city,elevation,latitude,longitude,state,utc_offset
tmy_id,,,,,,
700197,SELAWIK,26.24672,66.6,-160.0,AK,-9.0
